In [10]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install gdal

  Using cached GDAL-3.5.1.tar.gz (752 kB)
  Running setup.py clean for gdal
Failed to build gdal
    Running setup.py install for gdal: started
    Running setup.py install for gdal: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\alvaro\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\alvaro\\AppData\\Local\\Temp\\pip-install-fpfe8xho\\gdal_57bca3cbf92045f6807d93152b99688d\\setup.py'"'"'; __file__='"'"'C:\\Users\\alvaro\\AppData\\Local\\Temp\\pip-install-fpfe8xho\\gdal_57bca3cbf92045f6807d93152b99688d\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\alvaro\AppData\Local\Temp\pip-wheel-2yi891_g'
       cwd: C:\Users\alvaro\AppData\Local\Temp\pip-install-fpfe8xho\gdal_57bca3cbf92045f6807d93152b99688d\
  Complete output (118 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
 

In [5]:
import numpy as np
import pandas as pd
from scipy import stats

import warnings
warnings.filterwarnings(action='ignore')

import folium
import geopandas as gpd
KC_coord = (47.4081,-121.9949)
Seattle_coord = (47.6062,-122.3321)
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-notebook')
from yellowbrick.regressor import ResidualsPlot

import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OrdinalEncoder, OneHotEncoder

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
# specify parse date to avoid object dtype for dates
df = pd.read_csv('./data/kc_house_data_22.csv', parse_dates=['date']) 
df

In [ ]:
len(df)

In [ ]:
df.info()

# Set Zipcode

In [ ]:
df['address'][4]

In [ ]:
zipcode = []

for zips in df['address']:
    zipcode.append(int(zips[-20:-15]))
    
df['zipcode'] = zipcode
df.head()

# Fill NaNs and Drop Duplicates

### Duplicates

In [ ]:
# Check number of duplicate id's
df['id'].duplicated().sum()

In [ ]:
# Drop duplicates
df = df.sort_values('id', ascending = False).drop_duplicates(subset = 'id', keep = 'first')

In [ ]:
# Ensure that there are no longer duplicates
df.duplicated().sum()

### Heat Source

In [ ]:
df.isna().sum()

In [ ]:
# Check out sewer_system
print(df['sewer_system'].value_counts())

In [ ]:
df['heat_source'].fillna('Gas', inplace=True)

In [ ]:
df.isna().sum()

### Sewer System

In [ ]:
heat_count = print(df['sewer_system'].value_counts())

In [ ]:
df['sewer_system'].fillna('PUBLIC', inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['id', 'date'], axis=1, inplace=True)

# Train-Test Split

In [ ]:
y = df['price']
X = df.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.25)

# Helper Functions

ohe executes 2 transformations, test and train. data for any given input variable: First, we use OneHotEncoder to split up our categorical variables. Second, we run a linear regression on encoded values using .fit(). Then, it takes in a training df ('df'), a test df ('df2') and a column name. Next, it returns the training df and the test df concatenated with newly encoded columns. Lastly, we return a list of these newly encoded column names. 

In [ ]:
def ohe(df, df2, column):
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown="ignore")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_labels = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train, columns=col_labels, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
        test = df2[[col]]
        encoded_test = ohe.transform(test)
        col_labels = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_test = pd.DataFrame(encoded_test, columns=col_labels, index=df2.index)
        df2 = pd.concat([df2, encoded_test], axis=1)
        
    return df, df2, encoded_train.columns.tolist()

We run a Standard Scaler to scale all the values in the test data and training data so that our models results are in comparable units. Then, we score the scaled data to get the R2 to assess the strength of our model.

In [ ]:
def scale_score(x, y, x2, y2):
    scaler = StandardScaler()
    scaler.fit(x)
    X_train_scaled = scaler.transform(x)
    X_test_scaled = scaler.transform(x2)

    lr = LinearRegression()
    lr.fit(X_train_scaled,y_train)

    train_preds = lr.predict(X_train_scaled)
    test_preds = lr.predict(X_test_scaled)
    
    print(f"""training score: {r2_score(y_train, train_preds)}
    test score: {r2_score(y_test, test_preds)}""")
    
    return X_train_scaled, X_test_scaled, train_preds, test_preds

Uses price and predictions to calculate Mean Absolute Error (MAE), Mean Squared Error (MSE) and Root Mean Squared Error (RSME) for the training and testing data sets. We will compare these values to evaluate model performance.

In [ ]:
def get_train_test_metrics(y_train, train_preds,
               y_test, test_preds):
    """
    Prints different training and testing metrics, namely R2, MAE, MSE, RMSE
    """
    print("\nTraining Metrics:")
    print(f"R2: {r2_score(y_train, train_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_train, train_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_train, train_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_train, train_preds, squared=False):.3f}")
    
    print("\nTesting Metrics:")
    print(f"R2: {r2_score(y_test, test_preds):.3f}")
    print(f"Mean Absolute Error: {mean_absolute_error(y_test, test_preds):.3f}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, test_preds):.3f}")
    print(f"Root Mean Squared Error: {mean_squared_error(y_test, test_preds, squared=False):.3f}")

# Outliers

### Price

In [ ]:
# calculate 25, 75, iqr, whiskers for 'price'
price_q75, price_q25 = np.percentile(df['price'], [75, 25])
price_iqr = price_q75 - price_q25
price_min = price_q25 - (1.5 * price_iqr)
price_max = price_q75 + (1.5 * price_iqr)

print(f"""price_min: {price_min} 
price_max: {price_max}""")

In [ ]:
df = df[df['price'] >= price_min]
df = df[df['price'] <= price_max]

### Sqft Lot

In [ ]:
# calculate 25, 75, iqr, whiskers for 'sqft_lot'
#sqft_lot_q25, sqft_lot_q75 = np.percentile(df['sqft_lot'], [25, 75])
#sqft_lot_min = sqft_lot_q25 - (1.5 * sqft_lot_iqr)
#sqft_lot_max = sqft_lot_q75 + (1.5 * sqft_lot_iqr)


#print(f"""sqft_lot_min: {sqft_lot_min} 
#sqft_lot_max: {sqft_lot_max}""")

In [ ]:
# set df to include everything within the whiskers except the outliers
#df = df[df['sqft_lot'] >= sqft_lot_min]
#df = df[df['sqft_lot'] <= sqft_lot_max]

# Categorical Variables

In [ ]:
df.info()

In [ ]:
df.info()

### Bedrooms

In [ ]:
bed_count = df['bedrooms'].value_counts()
bed_count

In [ ]:
#df = df[df['bedrooms'] != 7]
#df = df[df['bedrooms'] != 8]
#df = df[df['bedrooms'] != 9]
#df = df[df['bedrooms'] != 10]
#df = df[df['bedrooms'] != 11]

### Condition

In [ ]:
condition_count = df['condition'].value_counts()
condition_count

In [ ]:
X_train, X_test, encoded_condition = ohe(X_train, X_test, ['condition'])

In [ ]:
X_train['condition'] = X_train['condition'].map({'Very Good': 5, 'Good': 4, 'Average': 3, 'Fair': 2, 'Poor': 1})

In [ ]:
X_test['condition'] = X_test['condition'].map({'Very Good': 5, 'Good': 4, 'Average': 3, 'Fair': 2, 'Poor': 1})

### Floors

In [ ]:
floor_count = print(df['floors'].value_counts())

In [ ]:
X_train, X_test, encoded_floors = ohe(X_train, X_test, ['floors'])

In [ ]:
#X_train['floors'] = X_train['floors'].map({1.0: 1, 1.5: 1, 2.0: 2, 2.5: 2, 3.0: 3, 3.5: 3, 4.0: 4})

In [ ]:
#X_test['floors'] = X_test['floors'].map({1.0: 1, 1.5: 1, 2.0: 2, 2.5: 2, 3.0: 3, 3.5: 3, 4.0: 4})

### Grade

In [ ]:
grade_count = print(df['grade'].value_counts())

In [ ]:
X_train, X_test, encoded_grade = ohe(X_train, X_test, ['grade'])

In [ ]:
#df = df[df['grade'] != '3 Poor']
#df = df[df['grade'] != '2 Substandard']
#df = df[df['grade'] != '13 Mansion']
#df = df[df['grade'] != '12 Luxury']

### Nuisance

In [ ]:
nuisance_count = print(df['nuisance'].value_counts())

In [ ]:
X_train, X_test, encoded_nuisance = ohe(X_train, X_test, ['nuisance'])

In [ ]:
X_train['nuisance'] = X_train['nuisance'].map({'NO': 0, 'YES': 1})

In [ ]:
X_test['nuisance'] = X_test['nuisance'].map({'NO': 0, 'YES': 1})

### View

In [ ]:
view_count = df['view'].value_counts()
view_count

In [ ]:
X_train['view'].fillna('NONE', inplace=True)
X_test['view'].fillna('NONE', inplace=True)

In [ ]:
X_train, X_test, encoded_view = ohe(X_train, X_test, ['view'])

In [ ]:
X_train['view'] = X_train['view'].map({'EXCELLENT': 5, 'GOOD': 4, 'AVERAGE': 3, 'FAIR': 2, 'NONE': 1})

In [ ]:
X_test['view'] = X_test['view'].map({'EXCELLENT': 5, 'GOOD': 4, 'AVERAGE': 3, 'FAIR': 2, 'NONE': 1})

### Waterfront

In [ ]:
waterfront_count = print(df['waterfront'].value_counts())

In [ ]:
X_train, X_test, encoded_waterfront = ohe(X_train, X_test, ['waterfront'])

In [ ]:
X_train['waterfront'] = X_train['waterfront'].map({'NO': 0, 'YES': 1})

In [ ]:
X_test['waterfront'] = X_test['waterfront'].map({'NO': 0, 'YES': 1})

### Zipcode

In [ ]:
zipcode_count = print(df['zipcode'].sort_values().value_counts())

In [ ]:
X_train, X_test, encoded_zipcode = ohe(X_train, X_test, ['zipcode'])

# Mutlitcollinearity

In [ ]:
sns.heatmap(df.corr());

In [ ]:
df.corr()

In [ ]:
df.corr()['price'].abs().sort_values(ascending=False)

In [ ]:
with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots(figsize=(14, 9))
    
    mask = np.triu(np.ones_like(df.corr(), dtype=bool))
    ax = sns.heatmap(abs(df.corr()),mask=mask,annot=True)
    fig.savefig('images/Corelation_Heatmap_22',bbox_inches="tight");

In [ ]:
df.drop(['sqft_above', 'sqft_lot'], axis=1, inplace=True)

# Models

In [ ]:
dummy = DummyRegressor()
dummy.fit(X_train, y_train)
basemodel_dummy_score = dummy.score(X_train, y_train)
test_dummy_score = dummy.score(X_test, y_test)

print(f"""basemodel_dummy_score: {basemodel_dummy_score}

test_dummy_score: {test_dummy_score}""")

### Model 1

In [ ]:
cols = ['sqft_living']

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 2

In [ ]:
cols = encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 3

In [ ]:
cols = encoded_grade

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 4

In [ ]:
cols = encoded_view

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 5

In [ ]:
cols = encoded_waterfront

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 6

In [ ]:
cols = encoded_grade + encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 7

In [ ]:
cols = encoded_grade + encoded_view + encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

### Model 8

In [ ]:
cols = ['sqft_living'] + encoded_grade + encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

In [ ]:
lr = LinearRegression()

visualizer = ResidualsPlot(lr)

visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show()  
plt.show()

### Model 9

In [ ]:
cols = ['sqft_living'] + encoded_grade + encoded_view + encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

In [ ]:
lr = LinearRegression()

visualizer = ResidualsPlot(lr)

visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show()  
plt.show()

### Model 10

In [ ]:
cols = ['sqft_living'] + encoded_grade + encoded_view + encoded_waterfront + encoded_zipcode

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

In [ ]:
lr = LinearRegression()

visualizer = ResidualsPlot(lr)

visualizer.fit(X_train_scaled, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test_scaled, y_test)  # Evaluate the model on the test data
visualizer.show()  
plt.show()

### Model 11

In [ ]:
cols = ['bathrooms', 'bedrooms', 'floors', 'sqft_living', 'yr_built'] + encoded_zipcode + encoded_grade + encoded_view + encoded_waterfront

y = y_train
x = X_train.filter(cols, axis=1)

y2 = y_test
x2 = X_test.filter(cols, axis=1)

In [ ]:
X_train_scaled, X_test_scaled, train_preds, test_preds = scale_score(x, y, x2, y2)

In [ ]:
get_train_test_metrics(y_train, train_preds, y_test, test_preds)

# Visualizations